In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
with open('healthcare-claims-prediction_kaggel/patient_data_train.json', 'r') as file:
    train_data_json = json.load(file)
with open('healthcare-claims-prediction_kaggel/patient_data_test.json', 'r') as file:
    test_data_json = json.load(file)

train_claims = pd.read_csv('healthcare-claims-prediction_kaggel/train.csv')

def normalize_and_clean_data(data):
    '''
    Normalizing and handling the data
    '''
    import warnings
    warnings.filterwarnings("ignore")
    df = pd.json_normalize(data)


    df["Conditions.HD"].fillna(df["Conditions.HD"].mode()[0],inplace=True)

    df["Conditions.HT"].fillna(df["Conditions.HD"].mode()[0],inplace=True)

    df["Conditions.DB"].fillna(df["Conditions.HD"].mode()[0],inplace=True)

    df["Conditions.AT"].fillna(df["Conditions.HD"].mode()[0],inplace=True)
    return df

train_data = normalize_and_clean_data(train_data_json)
test_data = normalize_and_clean_data(test_data_json)
    # As we already know, we have only 5 past years to handle the values
    # Now, we need to do the similar kind of analysis for conditions features as well.




train_data = pd.merge(train_data, train_claims, on='PatientID')

def fill_na_respectively(row, x, mean_val, feature_1, feature_2):
    
    if pd.isna(row[feature_2]):
        if pd.isna(row[feature_1]):
            row[feature_2] = mean_val
        else:
            row[feature_2] = row[feature_1] - x
    return row

respective_dif = {}

def find_dif(df, feature_1, feature_2):
    df_filtered = df.dropna(subset=[feature_1, feature_2])
    df_features2 = df.dropna(subset=[feature_2])
    name = feature_1 + feature_2
    df_filtered["difference"] = df_filtered[feature_1] - df_filtered[feature_2]
    respective_dif[name] = df_filtered["difference"].mean()



# Finding difference between 2022 and 2023
feature_1 = "Out patient costs.2023"
feature_2 = "Out patient costs.2022"
find_dif(train_data, feature_1, feature_2)

# Finding difference between 2021 and 2022
feature_1 = "Out patient costs.2022"
feature_2 = "Out patient costs.2021"
find_dif(train_data, feature_1, feature_2)


# Finding difference between 2021 and 2020
feature_1 = "Out patient costs.2021"
feature_2 = "Out patient costs.2020"
find_dif(train_data, feature_1, feature_2)

# Finding difference between 2020 and 2019
feature_1 = "Out patient costs.2020"
feature_2 = "Out patient costs.2019"
find_dif(train_data, feature_1, feature_2)

# print(respective_dif)


# Before filling rest of the cols, I will be filling the 2023 column
mean_values = train_data["Out patient costs.2023"].mean()
train_data["Out patient costs.2023"] = train_data["Out patient costs.2023"].fillna(mean_values)


# Filling null values in 2022 column
feature_1 = "Out patient costs.2023"
feature_2 = "Out patient costs.2022"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2021 column
feature_1 = "Out patient costs.2022"
feature_2 = "Out patient costs.2021"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)

# Filling null values in 2020 column
feature_1 = "Out patient costs.2021"
feature_2 = "Out patient costs.2020"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2019 column
feature_1 = "Out patient costs.2020"
feature_2 = "Out patient costs.2019"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Repeating the same for the test data

# Before filling rest of the cols, I will be filling the 2023 column
mean_values = train_data["Out patient costs.2023"].mean()
test_data["Out patient costs.2023"] = test_data["Out patient costs.2023"].fillna(mean_values)


# Filling null values in 2022 column
feature_1 = "Out patient costs.2023"
feature_2 = "Out patient costs.2022"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2021 column
feature_1 = "Out patient costs.2022"
feature_2 = "Out patient costs.2021"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)

# Filling null values in 2020 column
feature_1 = "Out patient costs.2021"
feature_2 = "Out patient costs.2020"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2019 column
feature_1 = "Out patient costs.2020"
feature_2 = "Out patient costs.2019"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)

trian_ID_col = train_data[["PatientID"]]
train_data.drop(["PatientID"],axis=1,inplace=True)
test_ID_col = test_data[["PatientID"]]
test_data.drop(["PatientID"], axis=1,inplace=True)


# converting Sex column into quantitative
from sklearn.preprocessing import LabelEncoder

sc = LabelEncoder()
sc.fit(train_data["Sex"])
train_data["Sex"] = sc.transform(train_data["Sex"])
test_data["Sex"] = sc.transform(test_data["Sex"])


x = train_data.drop(["TotalClaims"],axis=1)
y = train_data[["TotalClaims"]]



# These lines are making both the features in simlar order

temp = test_data[["Conditions.AT","Conditions.DB"]]
test_data.drop(["Conditions.AT","Conditions.DB"], axis=1, inplace=True)
test_data["Conditions.DB"] = temp["Conditions.DB"]
test_data["Conditions.AT"] = temp["Conditions.AT"]



from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)


# Sclaing the data

from sklearn.preprocessing import StandardScaler

sc= StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
test_data = sc.transform(test_data)





In [9]:
!pip install tensorflow


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.3 MB 10.6 MB/s eta 0:00:37
   ---------------------------------------- 2.0/390.3 MB 25.5 MB/s eta 0:00:16
   ---------------------------------------- 4.7/390.3 MB 37.4 MB/s eta 0:00:11
    --------------------------------------- 7.7/390.3 MB 44.8 MB/s eta 0:00:09
   - -------------------------------------- 10.1/390.3 MB 46.4 MB/s eta 0:00:09
   - -------------------------------------- 13.0/390.3 MB 65.6 MB/s eta 0:00:06
   - -------------------------------------- 15.4/390.3 MB 59.5 MB/s eta 0:00:07
   - -------------------------------------- 17.8/390.3 MB 54.7 MB/s eta 0:00:07
   -- ------------------------------------- 20.0/390.3 MB 54.4 MB/s eta 0:00:07
   -- ------------------------------------- 24.1/390.3 MB 59.

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))  

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

model.fit(x_train, y_train, epochs=300, batch_size=64, validation_split=0.25)

loss, mean_absolute_error = model.evaluate(x_test, y_test)

print(f'Test Mean Absolute Error: {mean_absolute_error}')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd

# Define the model architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
model.fit(x_train, y_train, epochs=300, batch_size=64, validation_split=0.25)

# Evaluate the model on the test set
loss, mean_absolute_error = model.evaluate(x_test, y_test)
print(f'Test Mean Absolute Error: {mean_absolute_error}')

# Generate predictions for the test data
predictions = model.predict(test_data_scaled)

# Save predictions to a CSV file
submission = pd.DataFrame({
    "PredictedValues": predictions.flatten()  # Flatten predictions to 1D if needed
})
submission.to_csv("submission.csv", index=False)

print("Predictions saved to submission.csv")





Epoch 1/300
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 1432191232.0000 - mean_absolute_error: 32935.5664 - val_loss: 814659328.0000 - val_mean_absolute_error: 24393.2305
Epoch 2/300
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 539245440.0000 - mean_absolute_error: 16793.0195 - val_loss: 171381888.0000 - val_mean_absolute_error: 9085.5391
Epoch 3/300
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 198881760.0000 - mean_absolute_error: 8586.3799 - val_loss: 97933240.0000 - val_mean_absolute_error: 6175.8662
Epoch 4/300
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 125390792.0000 - mean_absolute_error: 6349.0771 - val_loss: 86533048.0000 - val_mean_absolute_error: 5736.9761
Epoch 5/300
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 115339320.0000 - mean_absolute_error: 5817.6694 - val_loss: 79884936.0000 - val_mean_absolute_error: 5229.9521
Epoch 6/300
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 139137392.0000 - mean_absolute_error: 5697.9395 - val_loss: 78648032.0000

NameError: name 'test_data_scaled' is not defined